In [ ]:
# ! pip install bs4

In [5]:
import numpy as np
import bs4, requests
import pandas as pd

URL_TEMPLATE = "https://www.valmiki.iitk.ac.in/sloka?field_kanda_tid={kanda}&language=dv&field_sarga_value={sarga}"
KANDA = 6 #YUDDHAKANDA
OUTPUT_FILE = "../data/ramayana/data/yuddhakanda.csv"
SARGAS = np.arange(1,132)
URLS= [URL_TEMPLATE.format(kanda = KANDA, sarga = sarga) for sarga in SARGAS]
URLS[0]

'https://www.valmiki.iitk.ac.in/sloka?field_kanda_tid=6&language=dv&field_sarga_value=1'

In [8]:
def parse_html_from_url(url):
    print(url)
    try:
        # Send a GET request to the URL
        response = requests.get(url)
        response.raise_for_status()  # Check if the request was successful

        # Parse the HTML content using BeautifulSoup
        soup = bs4.BeautifulSoup(response.text, 'html.parser')

        shlokas = [elem.find_all(class_="field-content") for elem in soup.find_all(class_="views-row")]
        rows = [(shloka[0].text, shloka[2].text) for shloka in shlokas]
        return pd.DataFrame(rows, columns=('content', 'explanation'))

    except requests.exceptions.RequestException as e:
        print(f"Error fetching the URL: {e}")

# Example usage:
url_to_parse = URLS[0]
df = parse_html_from_url(url_to_parse)
df.head()

https://www.valmiki.iitk.ac.in/sloka?field_kanda_tid=6&language=dv&field_sarga_value=1


,content,explanation
0,"[Hearing Hanuman's report, Rama praises him an...","Having heard Hanuman's words, full of love, Ra..."
1,कृतं हनुमता कार्यं सुमहद्भुवि दुर्लभम् ।मनसापि...,"""Hanuman You have performed a mighty task inde..."
2,न हि तं परिपश्यामि यस्तरेत महार्णवम् ।अन्यत्र ...,"""None else than Garuda or Vayu have I seen to ..."
3,देवदानवयक्षाणां गन्धर्वोरगरक्षसाम् । अप्रधृष्य...,"""Indeed, who has the strength to return alive ..."
4,को विशेत्सुदुराधर्षां राक्षसैश्च सुरक्षिताम् ।...,"""Except for Hanuman, who is abounding in valou..."


In [13]:
dfs = [parse_html_from_url(url) for url in URLS]
final_df = pd.concat(dfs)
print(f"Number of sargas = {len(dfs)}, number of shlokas = {len(final_df)}")

Number of sargas = 131, number of shlokas = 5209


In [11]:
final_df.to_csv(OUTPUT_FILE, index=False)